In [35]:
# Importing the libraries
from tkinter import *
from tkinter import messagebox                           
import os            
import webbrowser
import numpy as np
import pandas as pd
from flask import Flask

In [36]:
#Importing the dataset
training_dataset = pd.read_csv('Training.csv')
test_dataset = pd.read_csv('Testing.csv')

#Slicing and Dicing the dataset to separate features from predictions
X = training_dataset.iloc[:, 0:132].values
Y = training_dataset.iloc[:, -1].values

# Dimensionality Reduction for removing redundancies
dimensionality_reduction = training_dataset.groupby(training_dataset['prognosis']).max()

#Encoding String values to integer constants
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Y)

#Splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

#Implementing the Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

#Saving the information of columns
cols = training_dataset.columns
cols = cols[:-1]

#Checking the Important features
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols




# Output in list

In [37]:

def execute_bot():

    print("Please reply with yes/Yes or no/No for the following symptoms") 
    def print_disease(node):
        #print(node)
        node = node[0]
        #print(len(node))
        val  = node.nonzero() 
        #print(val)
        disease = labelencoder.inverse_transform(val[0])
        return disease
    
    
    def tree_to_code(tree, feature_names):
        tree_ = tree.tree_
        #print(tree_)
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]
        #print("def tree({}):".format(", ".join(feature_names)))
        symptoms_present = []
        def recurse(node, depth):
            indent = "  " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                print(name + " ?")
                ans = input()
                ans = ans.lower()
                if ans == 'yes':
                    val = 1
                else:
                    val = 0
                if  val <= threshold:
                    recurse(tree_.children_left[node], depth + 1)
                else:
                    symptoms_present.append(name)
                    recurse(tree_.children_right[node], depth + 1)
            else:
                present_disease = print_disease(tree_.value[node])
                print( "You may have " +  present_disease )
                print()
                red_cols = dimensionality_reduction.columns 
                symptoms_given = red_cols[dimensionality_reduction.loc[present_disease].values[0].nonzero()]
                print("symptoms present  " + str(list(symptoms_present)))
                print()
                print("symptoms given "  +  str(list(symptoms_given)) )  
                print()
                print('The model suggests:')
                print()
                row = doctors[doctors['disease'] == present_disease[0]]
                print('Consult ', str(row['name'].values))
                print()
                print('Visit ', str(row['link'].values))
                #print(present_disease[0])
                
    
        recurse(0, 1)
    
    tree_to_code(classifier,cols)



# This section of code to be run after scraping the data

doc_dataset = pd.read_csv('doctors_dataset.csv', names = ['Name', 'Description'])


diseases = dimensionality_reduction.index
diseases = pd.DataFrame(diseases)

doctors = pd.DataFrame()
doctors['name'] = np.nan
doctors['link'] = np.nan
doctors['disease'] = np.nan

doctors['disease'] = diseases['prognosis']
 

doctors['name'] = doc_dataset['Name']
doctors['link'] = doc_dataset['Description']

record = doctors[doctors['disease'] == 'AIDS']
record['name']
record['link']

# Execute the bot and see it in Action
execute_bot()

Please reply with yes/Yes or no/No for the following symptoms
palpitations ?
yes
['You may have Hypoglycemia']

symptoms present  ['palpitations']

symptoms given ['vomiting', 'fatigue', 'anxiety', 'sweating', 'headache', 'nausea', 'blurred_and_distorted_vision', 'excessive_hunger', 'drying_and_tingling_lips', 'slurred_speech', 'irritability', 'palpitations']

The model suggests:

Consult  ['Dr. Jyoti Arora Monga']

Visit  ['https://www.practo.com/delhi/doctor/dr-jyoti-arora-ayurveda?specialization=Ayurveda&practice_id=693424']


# Run all below codes to have output in GUI

In [38]:
# Implementing the Visual Tree
from sklearn.tree import _tree

# Method to simulate the working of a Chatbot by extracting and formulating questions
def print_disease(node):
        #print(node)
        node = node[0]
        #print(len(node))
        val  = node.nonzero() 
        #print(val)
        disease = labelencoder.inverse_transform(val[0])
        return disease
def recurse(node, depth):
            global val,ans
            global tree_,feature_name,symptoms_present
            indent = "  " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                yield name + " ?"
                
#                ans = input()
                ans = ans.lower()
                if ans == 'yes':
                    val = 1
                else:
                    val = 0
                if  val <= threshold:
                    yield from recurse(tree_.children_left[node], depth + 1)
                else:
                    symptoms_present.append(name)
                    yield from recurse(tree_.children_right[node], depth + 1)
            else:
                strData=""
                present_disease = print_disease(tree_.value[node])
#                print( "You may have " +  present_disease )
#                print()
                strData="You may have :" +  str(present_disease)
               
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
                
                red_cols = dimensionality_reduction.columns 
                symptoms_given = red_cols[dimensionality_reduction.loc[present_disease].values[0].nonzero()]
#                print("Symptoms present  " + str(list(symptoms_present)))
#                print()
                strData="symptoms present:  " + str(list(symptoms_present))
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
#                print("Symptoms given "  +  str(list(symptoms_given)) )  
#                print()
                strData="symptoms given: "  +  str(list(symptoms_given))
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
                confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
#                print("onfidence level is " + str(confidence_level))
#                print()
                strData="confidence level is: " + str(confidence_level)
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
#                print('The model suggests:')
#                print()
                strData='The model suggests:'
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
                row = doctors[doctors['disease'] == present_disease[0]]
#                print('Consult ', str(row['name'].values))
#                print()
                strData='Consult '+ str(row['name'].values)
                QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
#                print('Visit ', str(row['link'].values))
                #print(present_disease[0])
                hyperlink = HyperlinkManager(QuestionDigonosis.objRef.txtDigonosis)
                strData='Visit '+ str(row['link'].values[0])
                def click1():
                    webbrowser.open_new(str(row['link'].values[0]))
                QuestionDigonosis.objRef.txtDigonosis.insert(INSERT, strData, hyperlink.add(click1))
                #QuestionDigonosis.objRef.txtDigonosis.insert(END,str(strData)+'\n')                  
                yield strData
        
def tree_to_code(tree, feature_names):
        global tree_,feature_name,symptoms_present
        tree_ = tree.tree_
        #print(tree_)
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]
        #print("def tree({}):".format(", ".join(feature_names)))
        symptoms_present = []   
#        recurse(0, 1)
    

def execute_bot():    
    tree_to_code(classifier,cols)



# This section of code to be run after scraping the data

doc_dataset = pd.read_csv('doctors_dataset.csv', names = ['Name', 'Description'])


diseases = dimensionality_reduction.index
diseases = pd.DataFrame(diseases)

doctors = pd.DataFrame()
doctors['name'] = np.nan
doctors['link'] = np.nan
doctors['disease'] = np.nan

doctors['disease'] = diseases['prognosis']


doctors['name'] = doc_dataset['Name']
doctors['link'] = doc_dataset['Description']

record = doctors[doctors['disease'] == 'AIDS']
record['name']
record['link']




# Execute the bot and see it in Action
#execute_bot()

1    https://www.practo.com/delhi/doctor/dr-54-gene...
Name: link, dtype: object

In [39]:
class Practo_managing_block:
      
    def __init__(self, text):
        self.text = text
        self.text.tag_config("hyper", foreground="brown", underline=1)
        self.text.tag_bind("hyper", "<Enter>", self._enter)
        self.text.tag_bind("hyper", "<Leave>", self._leave)
        self.text.tag_bind("hyper", "<Button-1>", self._click)

        self.reset()

    def reset(self):
        self.links = {}

    def add(self, action):
        # add an action to the manager.  returns tags to use in
        # associated text widget
        tag = "hyper-%d" % len(self.links)
        self.links[tag] = action
        return "hyper", tag

    def _enter(self, event):
        self.text.config(cursor="hand2")

    def _leave(self, event):
        self.text.config(cursor="")

    def _click(self, event):
        for tag in self.text.tag_names(CURRENT):
            if tag[:6] == "hyper-":
                self.links[tag]()
                return

# INTERFACE

In [40]:
 

#interface
class Diagnosis_of_questions(Frame):
    objIter=None
    objRef=None
    def __init__(self,master=None):
        master.title("Dr. Diagnosis")
        master.state("z")
        Diagnosis_of_questions.objRef=self
        super().__init__(master=master)
        self["bg"]="cyan"
        self.createWidget()
        self.iterObj=None



    def createWidget(self):
        self.lblQuestion = Label(self, text="Are you suffering with -->", width=20, bg="Cyan")
        self.lblQuestion.grid(row=0, column=0, rowspan=4)
    
        self.lblDigonosis = Label(self, text="Digonosis -->", width=20, bg="Cyan")
        self.lblDigonosis.grid(row=4, column=0, sticky="n", pady=5)
    
        self.txtQuestion = Text(self, width=120, height=4)
        self.txtQuestion.grid(row=0, column=1, rowspan=4, columnspan=20)
    
        self.varDiagonosis = StringVar()
        self.txtDigonosis = Text(self, width=120, height=14)
        self.txtDigonosis.grid(row=4, column=1, columnspan=20, rowspan=20, pady=5)
    
        self.btnNo = Button(self, text="No", width=12, bg="bisque", command=self.btnNo_Click)
        self.btnNo.grid(row=25, column=0)
    
        self.btnYes = Button(self, text="Yes", width=12, bg="bisque", command=self.btnYes_Click)
        self.btnYes.grid(row=25, column=1, columnspan=20, sticky="e")
    
        self.btnClear = Button(self, text="Clear", width=12, bg="bisque", command=self.btnClear_Click)
        self.btnClear.grid(row=27, column=0)
    
        self.btnStart = Button(self, text="Start", width=12, bg="bisque", command=self.btnStart_Click)
        self.btnStart.grid(row=27, column=1, columnspan=20, sticky="e")
    

        
    def btnNo_Click(self):
        global val,ans
        global val,ans
        ans='no'
        str1=Diagnosis_of_questions.objIter.__next__()
        self.txtQuestion.delete(0.0,END)
        self.txtQuestion.insert(END,str1+"\n")
        
        
        
        
    def btnYes_Click(self):
        global val,ans
        ans='yes'
        self.txtDigonosis.delete(0.0,END)
        str1=Diagnosis_of_questions.objIter.__next__()
        
        
        
        
    def btnClear_Click(self):
        self.txtDigonosis.delete(0.0,END)
        self.txtQuestion.delete(0.0,END)
        
        
        
    def btnStart_Click(self):
        execute_bot()
        self.txtDigonosis.delete(0.0,END)
        self.txtQuestion.delete(0.0,END)
        self.txtDigonosis.insert(END,"Please Click on Yes or No for the Above symptoms in Question")                  
        Diagnosis_of_questions.objIter=recurssive_process(0, 1)
        str1=Diagnosis_of_questions.objIter.__next__()
        self.txtQuestion.insert(END,str1+"\n")        
        



class MainForm(Frame):
    main_Root = None
    def destroyPackWidget(self, parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        MainForm.main_Root = master
        super().__init__(master=master)
        master.geometry("300x300")
        master.title("Account")
        self.createWidget()
        

        
    def createWidget(self):
        self.lblMsg=Label(self, text="Dr. Diagnosis", bg="lightblue", width="280", height="2", font=("Arial"))
        self.lblMsg.pack()
        self.btnLogin=Button(self, text="LOGIN", bg="lightgreen", height="1", width="15", font=("Arial",16),command = self.lblLogin_Click)
        self.btnLogin.pack()
        self.btnRegister=Button(self, text="REGISTER", bg="light green", height="1", width="15",font=("Arial",16), command = self.btnRegister_Click)
        self.btnRegister.pack()
        self.lblTeam=Label(self, text="Made by:", bg="white", width = "250", height = "1",padx=10, pady=10, font=("Arial", 16))
        self.lblTeam.pack()
        self.lblTeam1=Label(self, text="Tushar", bg="white", width = "250", height = "1", font=("Arial", 16))
        self.lblTeam1.pack()
        self.lblTeam2=Label(self, text="Abhinav Rai", bg="white", width = "250", height = "1", font=("Arial", 16))
        self.lblTeam2.pack() 
        self.lblTeam3=Label(self, text="Abhishek", bg="white", width = "250", height = "1", font=("Arial", 16))
        self.lblTeam3.pack()
        self.lblTeam4=Label(self, text="Gurleen", bg="white", width = "250", height = "1", font=("Arial", 16))
        self.lblTeam4.pack()         

        
    def lblLogin_Click(self):
        self.destroyPackWidget(MainForm.main_Root)
        frmLogin=Login(MainForm.main_Root)
        frmLogin.pack()
        
    def btnRegister_Click(self):
        self.destroyPackWidget(MainForm.main_Root)
        frmSignUp = SignUp(MainForm.main_Root)
        frmSignUp.pack()



        
class Login(Frame):
    main_Root=None
    def destroyPackWidget(self,parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        Login.main_Root=master
        super().__init__(master=master)
        master.title("Login")
        master.geometry("300x250")
        self.createWidget()
        
    def createWidget(self):
        self.lblMsg=Label(self, text="Please enter details below to login", width="300", font=("Arial", 16), padx=10, pady=10, bg="black", fg="white", )
        self.lblMsg.pack()
    
        self.username=Label(self, text="Username ", padx=10, pady=10, font=("Arial", 16))
        self.username.pack()
    
        self.username_verify = StringVar()
        self.username_login_entry = Entry(self, textvariable=self.username_verify)
        self.username_login_entry.pack()
    
        self.password=Label(self, text="Password ", padx=10, pady=10, font=("Arial", 16))
        self.password.pack()
    
        self.password_verify = StringVar()
        self.password_login_entry = Entry(self, textvariable=self.password_verify, show='*')
        self.password_login_entry.pack()
    
        self.btnLogin=Button(self, text="Login", width=9, height=1, font=("Arial", 16), bg="forest green", fg="white", command=self.btnLogin_Click)
        self.btnLogin.pack()

        
        

    def btnLogin_Click(self):
        username1 = self.username_login_entry.get()
        password1 = self.password_login_entry.get()

        
        list_of_files = os.listdir()
        if username1 in list_of_files:
            file1 = open(username1, "r")
            verify = file1.read().splitlines()
            if password1 in verify:
                messagebox.showinfo("Sucess","Login Sucessful")
                self.destroyPackWidget(Login.main_Root)
                frmQuestion = Diagnosis_of_questions(Login.main_Root)
                frmQuestion.pack()
            else:
                messagebox.showinfo("Failure", "Wrong Credentials - Try again")
        else:
            messagebox.showinfo("Failure", "User not found try from another user or sign up for new user")

            

class SignUp(Frame):
    main_Root=None
    def destroyPackWidget(self,parent):
        for e in parent.pack_slaves():
            e.destroy()
    def __init__(self, master=None):
        SignUp.main_Root=master
        master.title("Register")
        super().__init__(master=master)
        master.title("Register")
        master.geometry("300x250")
        self.createWidget()
        
        
        
    def createWidget(self):
        self.lblMsg=Label(self, text="Please enter the details below",width="300",font=("Arial", 16), padx=10, pady=10, bg="white", fg="black")
        self.lblMsg.pack()
        
        self.username_lable = Label(self, text="Username ",padx=10, pady=10,font=("Arial", 16))
        self.username_lable.pack()
        
        self.username = StringVar()
        self.username_entry = Entry(self, textvariable=self.username)
        self.username_entry.pack()
        

        self.password_lable = Label(self, text="Password ",padx=10, pady=10,font=("Arial", 16))
        self.password_lable.pack()
        self.password = StringVar()
        self.password_entry = Entry(self, textvariable=self.password, show='*')
        self.password_entry.pack()
        self.btnRegister=Button(self, text="Register",font=("Arial", 16), bg="forest green", fg="white", command=self.register_user)
        self.btnRegister.pack()


        
    def register_user(self):
        file = open(self.username_entry.get(), "w")
        file.write(self.username_entry.get() + "\n")
        file.write(self.password_entry.get())
        file.close()
        
        self.destroyPackWidget(SignUp.main_Root)
        
        self.lblSucess=Label(root, text="Registration Success", fg="green", font=("Arial", 16))
        self.lblSucess.pack()
        
        self.btnSucess=Button(root, text="Click Here to proceed", command=self.btnSucess_Click)
        self.btnSucess.pack()
        
        
        
    def btnSucess_Click(self):

        self.destroyPackWidget(SignUp.main_Root)
        frmQuestion = Diagnosis_of_questions(SignUp.main_Root)

        frmQuestion.pack()


        

root = Tk()

frmMainForm=MainForm(root)
frmMainForm.pack()
root.mainloop()